In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager


import pandas as pd

In [2]:
# Set up Chrome options if needed
chrome_options = webdriver.ChromeOptions()
# Add any desired options here, e.g., headless mode
# chrome_options.add_argument('--headless')

# Create a Service object with the path to the ChromeDriver
service = Service(ChromeDriverManager().install())

# Initialize the WebDriver with the Service object and options
driver = webdriver.Chrome(service=service, options=chrome_options)

# Your automation code here

driver.quit()


KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import re

In [ ]:

def setup_driver():
    """Set up and return a configured Chrome WebDriver."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (no UI)
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-notifications")
    
    # Initialize the Chrome driver
    driver = webdriver.Chrome(options=chrome_options)
    return driver


In [ ]:

def get_shop_details(stores):
    """Visit each shop page and extract detailed information."""
    driver = setup_driver()
    detailed_data = []
    # stores = pd.read_csv("sunway_pyramid_shops.csv")

    i = 0  # Counter for testing, remove later
    
    for _, store in stores.iterrows():

        shop_url = store.get("url", "")
        if not shop_url:
            continue  # Skip if URL is missing

        try:
            driver.get(shop_url)
            
            # Wait for the shop page to load
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".logocontainer-shopview"))
            )
            
            # Extract shop details
            try:
                logo_element = driver.find_element(By.CSS_SELECTOR, ".logocontainer-shopview img")
                logo = logo_element.get_attribute("src")
            except:
                logo = "N/A"
            
            # Extract shop category
            try:
                category = driver.find_element(By.CSS_SELECTOR, ".__rounded-pill").text.strip()
            except:
                category = "N/A"
                
            # Extract shop description
            try:
                description = driver.find_element(By.CSS_SELECTOR, ".font-normal.long-shop-text-scroll").text.strip()
            except:
                description = "N/A"
                
            # Extract location
            try:
                location = driver.find_element(By.XPATH, "//a[@href='#mapZone']").text.strip()
            except:
                location = "N/A"

            detailed_data.append({
                "name": store["name"],
                "img":store["img"],
                "url": shop_url,
                "logo": logo,
                "category": category,
                "description": description,
                "location": location
            })
            
        except Exception as e:
            print(f"Error scraping details from {shop_url}: {e}")
    
    driver.quit()
    
    # Create DataFrame and save to CSV
    if detailed_data:
        detailed_df = pd.DataFrame(detailed_data)
        detailed_df.to_csv("sunway_pyramid_shop_details.csv", index=False)
        return detailed_df
    
    return None



In [ ]:

async def scrape_sunway_pyramid_directory():
    """Scrape the Sunway Pyramid mall directory."""
    url = "https://www.sunwaypyramid.com/directory"
    driver = setup_driver()
    
    try:
        print("Accessing the Sunway Pyramid directory...")
        driver.get(url)
        
        # Wait for the page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".btn-outline-danger"))
        )

        # Allow some time for initial content to load
        time.sleep(3)

        # Click "Load More" until it disappears
        while True:
            try:
                # Find all buttons with class ".btn-outline-danger"
                buttons = driver.find_elements(By.CSS_SELECTOR, ".btn-outline-danger")
                
                # Click only the button that contains "Load More"
                load_more_clicked = False
                for button in buttons:
                    if "Load More" in button.text:
                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)  # Scroll into view
                        time.sleep(1)  # Allow time for scrolling
                        button.click()
                        time.sleep(2)  # Wait for new items to load
                        load_more_clicked = True
                        break  # Stop looking for other buttons once clicked
                
                # If no button was clicked, exit loop
                if not load_more_clicked:
                    print("No more 'Load More' button. All items loaded.")
                    break  

            except Exception as e:
                print(f"Error clicking 'Load More': {e}")
                break  # Exit loop if an error occurs


        # Extract all shop elements
        shop_data = []
        cards = driver.find_elements(By.CSS_SELECTOR, ".card")

        for card in cards:
            try:
                # Extract shop name
                title = card.find_element(By.CSS_SELECTOR, '.text-truncate').text.strip()
                directory_route=card.find_element(By.TAG_NAME, 'a').get_attribute("href")
                # Extract image URL
                img_url = card.find_element(By.TAG_NAME, 'img').get_attribute("src")

                # Append to list
                shop_data.append({"name": title, "img": img_url, "url":directory_route})
            except Exception as e:
                print(f"Error processing card: {e}")

        # Convert to DataFrame and save as CSV
        df = pd.DataFrame(shop_data)
        # df.to_csv("sunway_pyramid_shops.csv", index=False)
        print(f"Successfully scraped {len(shop_data)} shops from Sunway Pyramid directory.")

        return df

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()


In [41]:

# Scrape the main directory
shops_df = await scrape_sunway_pyramid_directory()

Accessing the Sunway Pyramid directory...
No more 'Load More' button. All items loaded.
Successfully scraped 822 shops from Sunway Pyramid directory.


In [42]:
get_shop_details(shops_df)

,name,img,url,logo,category,description,location
0,1 MOBILE STORE,https://www.sunwaypyramid.com/static/shops/2eb...,https://www.sunwaypyramid.com/directory/1-mobi...,https://www.sunwaypyramid.com/static/shops/a2c...,Digital Lifestyle,1 Mobile Store (also known as One Specialist M...,F1.39 & F1.39A (First Floor)
1,1 Specialist Mobile,https://www.sunwaypyramid.com/static/shops/273...,https://www.sunwaypyramid.com/directory/1-spec...,https://www.sunwaypyramid.com/static/shops/ed2...,Digital Lifestyle,Retailing of gadget accessories and related pr...,F1.38A (First Floor)
2,100 Vision,https://www.sunwaypyramid.com/static/shops/66e...,https://www.sunwaypyramid.com/directory/100-vi...,https://www.sunwaypyramid.com/static/shops/315...,Eyewear,"Retailing of eyewear, sunglasses and related p...",F1.101 (First Floor)
3,108 Matcha Saro,https://www.sunwaypyramid.com/static/shops/d82...,https://www.sunwaypyramid.com/directory/108-ma...,https://www.sunwaypyramid.com/static/shops/630...,Food & Beverages,108 Matcha Saro celebrates the rich tradition ...,GF.122C (Ground Floor)
4,30DEGREES,https://www.sunwaypyramid.com/static/shops/1f8...,https://www.sunwaypyramid.com/directory/30degrees,https://www.sunwaypyramid.com/static/shops/fb8...,Fashion,"Retailing of ladies sports fashion, lingerie a...",LG1.02 (Lower Ground 1 Floor)
...,...,...,...,...,...,...,...
817,iORA,https://www.sunwaypyramid.com/static/shops/03a...,https://www.sunwaypyramid.com/directory/iora,https://www.sunwaypyramid.com/static/shops/07e...,Fashion,The heart of the business is well-made apparel...,"LG1.06, 07 & 10 (Lower Ground 1 Floor)"
818,llao llao,https://www.sunwaypyramid.com/static/shops/14c...,https://www.sunwaypyramid.com/directory/llao-llao,https://www.sunwaypyramid.com/static/shops/96d...,Food & Beverages,Retailing of frozen yogurt and related products,F1.49 (First Floor)
819,myEureka Snack Bar,https://www.sunwaypyramid.com/static/shops/5bc...,https://www.sunwaypyramid.com/directory/myeure...,https://www.sunwaypyramid.com/static/shops/97c...,Food & Beverages,Retailing of assorted flavoured popcorn.,F1.100D (First Floor)
820,nononsense coffee,https://www.sunwaypyramid.com/static/shops/bc9...,https://www.sunwaypyramid.com/directory/nonons...,https://www.sunwaypyramid.com/static/shops/5c5...,Food & Beverages,"Instilled with the elements of an Otaku Room, ...",LG1.33 - 37 (inside Atmos) (Lower Ground 1 Floor)


In [34]:
# get_shop_details()

,name,url,logo,category,description,location
0,1 MOBILE STORE,https://www.sunwaypyramid.com/directory/1-mobi...,https://www.sunwaypyramid.com/static/shops/a2c...,Digital Lifestyle,1 Mobile Store (also known as One Specialist M...,F1.39 & F1.39A (First Floor)
1,1 Specialist Mobile,https://www.sunwaypyramid.com/directory/1-spec...,https://www.sunwaypyramid.com/static/shops/ed2...,Digital Lifestyle,Retailing of gadget accessories and related pr...,F1.38A (First Floor)
2,100 Vision,https://www.sunwaypyramid.com/directory/100-vi...,https://www.sunwaypyramid.com/static/shops/315...,Eyewear,"Retailing of eyewear, sunglasses and related p...",F1.101 (First Floor)
3,108 Matcha Saro,https://www.sunwaypyramid.com/directory/108-ma...,https://www.sunwaypyramid.com/static/shops/630...,Food & Beverages,108 Matcha Saro celebrates the rich tradition ...,GF.122C (Ground Floor)
4,30DEGREES,https://www.sunwaypyramid.com/directory/30degrees,https://www.sunwaypyramid.com/static/shops/fb8...,Fashion,"Retailing of ladies sports fashion, lingerie a...",LG1.02 (Lower Ground 1 Floor)
...,...,...,...,...,...,...
817,iORA,https://www.sunwaypyramid.com/directory/iora,https://www.sunwaypyramid.com/static/shops/07e...,Fashion,The heart of the business is well-made apparel...,"LG1.06, 07 & 10 (Lower Ground 1 Floor)"
818,llao llao,https://www.sunwaypyramid.com/directory/llao-llao,https://www.sunwaypyramid.com/static/shops/96d...,Food & Beverages,Retailing of frozen yogurt and related products,F1.49 (First Floor)
819,myEureka Snack Bar,https://www.sunwaypyramid.com/directory/myeure...,https://www.sunwaypyramid.com/static/shops/97c...,Food & Beverages,Retailing of assorted flavoured popcorn.,F1.100D (First Floor)
820,nononsense coffee,https://www.sunwaypyramid.com/directory/nonons...,https://www.sunwaypyramid.com/static/shops/5c5...,Food & Beverages,"Instilled with the elements of an Otaku Room, ...",LG1.33 - 37 (inside Atmos) (Lower Ground 1 Floor)
